<a href="https://colab.research.google.com/github/abxda/python/blob/main/UP_Semana_4_Viernes_y_S%C3%A1bado_03_ENIGH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from tqdm import tqdm
import requests
from zipfile import ZipFile
import time
import os
import shutil
import numpy as np

In [ ]:
#Funciones de apoyo
def download(url,dir):
    time.sleep(5)
    chunk_size = 1024
    r = requests.get(url, stream = True)
    total_size = int(r.headers['content-length'])
    filename = url.split('/')[-1]
    with open(dir+filename, 'wb') as f:
        for data in tqdm(iterable = r.iter_content(chunk_size = chunk_size), total = total_size/chunk_size, unit = 'KB'):
            f.write(data)

In [ ]:
os.makedirs("./inegi/enigh", exist_ok=True)

In [ ]:
# Descarga los Datos de de ENIGH
directory= "./inegi/enigh/"
datos = "https://www.inegi.org.mx/contenidos/programas/enigh/nc/2022/datosabiertos/conjunto_de_datos_enigh_ns_2022_csv.zip"
download(datos,directory)

In [ ]:
datos_zip = "./inegi/enigh/conjunto_de_datos_enigh_ns_2022_csv.zip"

with ZipFile(datos_zip, 'r') as zip:
    zip.extractall(directory)

In [ ]:
concentradohogar = pd.read_csv("./inegi/enigh/conjunto_de_datos_concentradohogar_enigh2022_ns/conjunto_de_datos/conjunto_de_datos_concentradohogar_enigh2022_ns.csv", low_memory=False)
gastoshogar = pd.read_csv("./inegi/enigh/conjunto_de_datos_gastoshogar_enigh2022_ns/conjunto_de_datos/conjunto_de_datos_gastoshogar_enigh2022_ns.csv")
geo_entidad = pd.read_csv("./inegi/enigh/conjunto_de_datos_concentradohogar_enigh2022_ns/catalogos/ubica_geo.csv", encoding='iso-8859-1')[["entidad","desc_ent"]].drop_duplicates()

In [ ]:
gastoshogar

In [ ]:
concentradohogar

In [ ]:
geo_entidad

In [ ]:
catalogo_gastos = pd.read_csv("./inegi/enigh/conjunto_de_datos_gastospersona_enigh2022_ns/catalogos/gastos.csv",encoding='iso-8859-1')

In [ ]:
catalogo_gastos

In [ ]:
gastoshogar["gasto_tri"] = pd.to_numeric(gastoshogar["gasto_tri"], downcast="float", errors='coerce')
aceite_cocina = gastoshogar.query("clave=='A095' or clave=='A096'")

In [ ]:
aceite_cocina

In [ ]:
aceite_agregado_hogar = aceite_cocina.groupby(['folioviv','foliohog']).agg({'gasto_tri':['sum']})

In [ ]:
aceite_agregado_hogar.columns = aceite_agregado_hogar.columns.droplevel()
aceite_agregado_hogar = aceite_agregado_hogar.reset_index()
suma_gasto_aceite = aceite_agregado_hogar[['sum']].values
aceite_agregado_hogar = aceite_agregado_hogar[['folioviv','foliohog']]
aceite_agregado_hogar['suma_gasto_aceite'] = suma_gasto_aceite

In [ ]:
aceite_agregado_hogar

In [ ]:
concentradohogar = concentradohogar.merge(aceite_agregado_hogar, how='left', left_on=['folioviv','foliohog'], right_on = ['folioviv','foliohog'])


In [ ]:
concentradohogar

In [ ]:
concentradohogar['deciles'] = pd.qcut(concentradohogar['ing_cor'],q = 10, labels = False)

In [ ]:
concentradohogar['ubica_geo'] = concentradohogar['ubica_geo']*0.001
concentradohogar['ubica_geo'] = concentradohogar['ubica_geo'].astype(int)
concentradohogar_deciles_geo = concentradohogar.merge(geo_entidad, how='left', left_on=['ubica_geo'], right_on = ['entidad'])

In [ ]:
concentradohogar_deciles_geo

In [ ]:
np.sum(concentradohogar_deciles_geo.query('suma_gasto_aceite > 0')[["factor"]].values)

In [ ]:
np.sum(concentradohogar_deciles_geo[["factor"]].values)

In [ ]:
np.mean(concentradohogar_deciles_geo.query('suma_gasto_aceite > 0')[["suma_gasto_aceite"]].values)

In [ ]:
gasto_promedio_estatal = concentradohogar_deciles_geo.query('suma_gasto_aceite > 0').groupby(['desc_ent']).agg({'suma_gasto_aceite':['mean']})
gasto_promedio_estatal = gasto_promedio_estatal.reset_index()
media_estatal = gasto_promedio_estatal[['suma_gasto_aceite']].values
gasto_promedio_estatal = gasto_promedio_estatal[['desc_ent']]
gasto_promedio_estatal['media_estatal'] = media_estatal
gasto_promedio_estatal.sort_values("media_estatal", ascending=False)

In [ ]:
np.mean(concentradohogar_deciles_geo.query('suma_gasto_aceite > 0')[["ing_cor"]].values)

In [ ]:
56525/3

In [ ]:
ingreso_promedio_estatal = concentradohogar_deciles_geo.query('suma_gasto_aceite > 0').groupby(['desc_ent']).agg({'ing_cor':['mean']})
ingreso_promedio_estatal = ingreso_promedio_estatal.reset_index()
ing_media_estatal = ingreso_promedio_estatal[['ing_cor']].values
ingreso_promedio_estatal = ingreso_promedio_estatal[['desc_ent']]
ingreso_promedio_estatal['media_estatal'] = ing_media_estatal
ingreso_promedio_estatal.sort_values("media_estatal", ascending=False)

In [ ]:
gasto_promedio_educacion = concentradohogar_deciles_geo.query('suma_gasto_aceite > 0').groupby(['educa_jefe']).agg({'suma_gasto_aceite':['mean'], 'factor':['sum']})
gasto_promedio_educacion = gasto_promedio_educacion.reset_index()
media_gasto_educacion = gasto_promedio_educacion[['suma_gasto_aceite']]
hogares  = gasto_promedio_educacion[['factor']]
gasto_promedio_educacion = gasto_promedio_educacion[['educa_jefe']]
gasto_promedio_educacion['media_gasto_educacion'] = media_gasto_educacion
gasto_promedio_educacion['total_hogares'] = hogares
gasto_promedio_educacion